# This notebook is looking at the commissioning designs for Cool Stars
These are detailed in an email sent out by Joe Burchett on 12/17/21 to the commissioning list.

Data files for review can be found here: https://wiki.sdss.org/display/COM/Commissioning+design+review

 catalog of selected targets and designs from the query

    *_targs_*.fits  

A catalog of all targets in each design that contains targets meeting selection criteria

    *_designDetail.fits  

## Notes

12255 has 13 dwarfs (4 cool)
12217 has 16 dwarfs (3 cool 1 very cool 3200K)
12831 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.table import Table
from astropy import modeling
import matplotlib
%matplotlib inline 

In [2]:
target_tab = Table.read('teff-lt-6000_targs_commwm-5-31-59.fits')
target_tab

design_id,field_id,racen,deccen,ra,dec,teff,feh,logg,j,h,k,label,program,carton,num_targs
int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6,bytes13,bytes25,int64
12987,19170,101.661685072784,68.8638559646908,103.08136935569146,67.90775515942933,4519.4497,0.19791844,nan,8.17,7.564,7.437,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,101.4354399470212,69.88223302200862,4962.4517,-0.47284156,nan,8.641,8.127,8.018,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,101.54839574517766,69.99176445744277,4399.1133,-0.54978156,nan,10.711,10.019,9.884,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,101.03022777989794,70.04260187546524,4498.814,-0.17318155,nan,10.293,9.669,9.534,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,103.68465685169474,67.80896514266378,4068.4392,-0.108281545,nan,9.343,8.606,8.433,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,103.05082934458275,68.48731030668691,4608.3994,0.38572568,nan,10.494,9.944,9.789,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,102.52357385315314,68.3115623584817,4698.328,-0.22667809,nan,9.837,9.259,9.127,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,101.16008181162343,67.69510578855704,4758.641,0.12014845,nan,9.56,9.02,8.888,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90
12987,19170,101.661685072784,68.8638559646908,100.83648553176863,67.60223208604896,4849.4937,-0.42747155,nan,10.411,9.82,9.746,APOGEE,mwm_tessrgb,mwm_tessrgb_core,90


In [3]:
field_tab = target_tab[target_tab['design_id'] == 12798]
#field_tab.show_in_notebook()
#field_tab[np.isfinite(field_tab['logg'])].show_in_notebook()
field_tab[np.isfinite(field_tab['logg']) & (field_tab['teff'] < 3800)].show_in_notebook()

idx,design_id,field_id,racen,deccen,ra,dec,teff,feh,logg,j,h,k,label,program,carton,num_targs
0,12798,19092,123.996668658484,47.2267531776921,123.68162672239548,46.843160420821505,3433.3716,0.157911,4.85599,10.524,9.893,9.625,APOGEE,mwm_galactic,mwm_galactic_core,67
1,12798,19092,123.996668658484,47.2267531776921,123.20016901796214,47.719972735109096,3610.612,-0.09459164,4.77835,10.936,10.341,10.107,APOGEE,mwm_snc,mwm_snc_100pc_apogee,67
2,12798,19092,123.996668658484,47.2267531776921,124.04492344764944,46.86995278832162,3554.315,-0.023227,4.83091,11.287,10.691,10.439,APOGEE,mwm_snc,mwm_snc_100pc_apogee,67
3,12798,19092,123.996668658484,47.2267531776921,123.05956699808773,46.577122197780035,3794.4026,-0.32001898,4.74639,11.293,10.705,10.473,APOGEE,mwm_snc,mwm_snc_100pc_apogee,67
4,12798,19092,123.996668658484,47.2267531776921,123.95342197283436,46.02178057758114,3735.0244,-0.126369,4.75272,9.78,9.16,8.944,APOGEE,mwm_cb,mwm_cb_gaiagalex_apogee,67


In [4]:
targ_list = list()
rgb_list = list()
logg_list = list()
subg_list = list()
dwarf_list = list()
cool_list = list()

design_arr = np.unique(target_tab['design_id'])
for design in design_arr:
    design_tab = target_tab[target_tab['design_id'] == design]
    num_targ = len(design_tab)
    targ_list.append(num_targ)
    num_rgb = len(design_tab[design_tab['program'] == 'mwm_tessrgb'])
    rgb_list.append(num_rgb)
    num_logg = len(design_tab[np.isfinite(design_tab['logg'])& (design_tab['label'] == 'APOGEE')])
    logg_list.append(num_logg)
    num_subg = len(design_tab[(design_tab['logg'] < 4.1) & (design_tab['logg'] >= 3.5) & (design_tab['label'] == 'APOGEE')])
    subg_list.append(num_subg)
    num_dwarf = len(design_tab[(design_tab['logg'] > 4.1) & (design_tab['label'] == 'APOGEE')])
    dwarf_list.append(num_dwarf)
    num_cool = len(design_tab[(design_tab['logg'] > 4.1) & (design_tab['teff'] < 3800) & 
                   (design_tab['label'] == 'APOGEE')])
    cool_list.append(num_cool)
    #print("Design: {} Num: {} RGB: {} Log g: {} Subg: {} Cool: {}".format(design, num_targ,num_rgb,num_logg
    #                                                                     ,num_subg, num_cool))

summary_tab = Table()
summary_tab['design_id'] = design_arr
summary_tab['num'] = targ_list
summary_tab['rgb'] = rgb_list
summary_tab['logg'] = logg_list
summary_tab['subg'] = subg_list
summary_tab['dwarf'] = dwarf_list
summary_tab['cool'] = cool_list

In [5]:
summary_tab.sort('cool')
summary_tab.reverse()
best_tab = summary_tab[(summary_tab['subg'] >= 3) & (summary_tab['rgb'] > 20) & (summary_tab['cool'] >= 3)]
best_tab.sort(['cool','subg'])
best_tab.reverse()
best_tab

design_id,num,rgb,logg,subg,dwarf,cool
int64,int64,int64,int64,int64,int64,int64
12798,67,34,28,6,22,5
12781,63,32,30,5,25,5
12853,65,31,30,4,26,5
12875,76,37,38,7,31,4
12780,63,41,19,3,16,4
12788,56,23,30,3,27,4
13467,59,23,30,9,21,3
13135,67,40,26,3,23,3
12787,64,32,27,3,23,3


In [6]:
np.sum(best_tab['num'])

580

In [7]:
mask = np.isin(target_tab['design_id'],best_tab['design_id'])
best_target_tab = target_tab[mask]
best_target_tab

design_id,field_id,racen,deccen,ra,dec,teff,feh,logg,j,h,k,label,program,carton,num_targs
int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6,bytes13,bytes25,int64
12875,19124,126.502324661715,56.2376095398739,126.9436698238534,56.258525768343354,5413.0,-0.40452,4.32643,10.652,10.428,10.385,APOGEE,ops_std,ops_std_apogee,76
12875,19124,126.502324661715,56.2376095398739,127.09081761542194,55.80708489732809,4861.603,-0.25985247,nan,7.995,7.5,7.379,APOGEE,mwm_tessrgb,mwm_tessrgb_core,76
12875,19124,126.502324661715,56.2376095398739,124.83932016373592,56.45200140277449,5605.6665,0.00336762,nan,7.79,7.523,7.449,APOGEE,ops_std,ops_std_apogee,76
12875,19124,126.502324661715,56.2376095398739,126.3531390064832,56.96496746843342,5964.3247,-0.031054,4.43401,9.457,9.219,9.162,APOGEE,mwm_planet,mwm_tess_planet,76
12875,19124,126.502324661715,56.2376095398739,127.90783308077027,55.71140119027075,5309.811,0.367551,4.29574,10.127,9.802,9.735,APOGEE,mwm_snc,mwm_snc_250pc_apogee,76
12875,19124,126.502324661715,56.2376095398739,128.7619190832328,56.18820356118905,4987.23,-0.340639,4.65858,9.326,8.892,8.76,APOGEE,mwm_snc,mwm_snc_100pc_apogee,76
12875,19124,126.502324661715,56.2376095398739,127.6019757857465,55.65506385466518,3078.0,-0.35499,4.09058,10.976,10.772,10.711,APOGEE,ops_std,ops_std_apogee,76
12875,19124,126.502324661715,56.2376095398739,125.92238137497014,56.976055431241335,5006.463,-0.62360156,nan,9.847,9.411,9.3,APOGEE,mwm_tessrgb,mwm_tessrgb_core,76
12875,19124,126.502324661715,56.2376095398739,125.04681788809465,56.00649159695056,4806.064,-0.045778997,4.40787,9.747,9.248,9.147,APOGEE,mwm_snc,mwm_snc_250pc_apogee,76


In [8]:
best_target_tab.write('best_field_targets_cool_stars.fits')

## Detailed view

In [9]:
detail_tab = Table.read('teff-lt-6000_designDetail.fits')
detail_tab

design_id,field,exposure,design_mode,mugatu_version,run_on,pk,field_id,racen,deccen,position_angle,cadence,observatory,version,catalogid,dec,epoch,pmdec,pmra,ra,parallax,bp,g,h,i,z,r,rp,gaia_g,j,k,optical_prov,carton_to_target,label,carton
int64,int64,int64,bytes15,bytes5,bytes10,int64,int64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes26,float64,bytes6,bytes27
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610820798,58.89606524843866,2015.5,nan,nan,136.2818308550186,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610820806,58.97756301972696,2015.5,nan,nan,136.27463312368974,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610820841,59.233805934166284,2015.5,nan,nan,136.5829889160248,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610820885,58.856051350557486,2015.5,nan,nan,135.81032162249966,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610820936,59.056082018575225,2015.5,nan,nan,135.19545114888848,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610821070,59.36112808014974,2015.5,nan,nan,136.00372536074696,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610821460,59.03534259987552,2015.5,nan,nan,133.41430971716605,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610821479,59.19382468095364,2015.5,nan,nan,133.459120971903,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss
73,80,0,bright_time_eng,1.0.4,2021-12-01,80,16075,135.0,60.0,0.0,1,0,84,7610821632,59.16124452201103,2015.5,nan,nan,134.81444168501943,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,APOGEE,ops_sky_boss


In [10]:
mask = np.isin(detail_tab['design_id'],best_tab['design_id'])
best_detail_tab = detail_tab[mask]
best_detail_tab

design_id,field,exposure,design_mode,mugatu_version,run_on,pk,field_id,racen,deccen,position_angle,cadence,observatory,version,catalogid,dec,epoch,pmdec,pmra,ra,parallax,bp,g,h,i,z,r,rp,gaia_g,j,k,optical_prov,carton_to_target,label,carton
int64,int64,int64,bytes15,bytes5,bytes10,int64,int64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes26,float64,bytes6,bytes27
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920011983,43.639797656162926,2015.5,-0.832874,-4.58721,125.97706084546586,0.72752,16.29005,16.18859,14.375,15.39281,15.489831,15.798201,15.195149,15.827515,14.862,14.422,sdss_psfmag_from_csc,541404611.0,BOSS,bhm_csc_boss_bright
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920012393,43.6224932588852,2015.5,nan,nan,126.054819021414,nan,nan,18.156498,15.471,17.856194,17.884903,17.783726,nan,19.175837,15.978,14.735,sdss_psfmag_from_csc,541404612.0,BOSS,bhm_csc_boss_bright
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920074584,42.96310592097383,2015.5,-1.167,-0.0191511,124.7940889975821,0.0345864,15.613192,16.258123,8.945,12.610172,11.561849,14.646992,12.0390625,13.422412,9.857,8.597,sdss_psfmag,473952671.0,APOGEE,mwm_galactic_core
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021599488775281,42.988801298607825,2015.5,-38.95,15.92,126.10106670587993,nan,nan,11.29599,9.401,10.626916,10.995631,10.768576,nan,nan,9.729,9.338,sdss_psfmag,479058203.0,APOGEE,mwm_galactic_core
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920124731,43.34309579156566,2015.5,14.8843,9.28793,127.71619063911224,nan,15.345653,15.947467,10.792,13.320891,nan,14.460939,12.898236,14.067732,11.367,10.533,sdss_psfmag,497016535.0,APOGEE,mwm_snc_100pc_apogee
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920125224,43.205600791719036,2015.5,-17.1087,-132.58,127.62647081167079,13.2893,15.008599,15.593111,10.504,13.04619,nan,14.148439,12.637184,13.74535,11.144,10.281,sdss_psfmag,497016537.0,APOGEE,mwm_snc_100pc_apogee
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920125588,43.45759441447871,2015.5,-60.37,-29.31,126.7087892082817,nan,14.77618,15.40934,10.44,12.906922,nan,13.927421,12.509611,13.856598,11.04,10.201,sdss_psfmag,497016548.0,APOGEE,mwm_snc_100pc_apogee
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920181283,42.10843119028469,2015.5,-18.4912,-6.31681,126.47624168165383,18.0992,13.64866,14.225892,9.494,13.439476,nan,12.79418,11.495725,12.5325365,10.124,9.272,sdss_psfmag,497016614.0,APOGEE,mwm_snc_100pc_apogee
12780,3052,0,bright_time,1.0.4,2021-12-03,3052,19084,126.393242406942,43.404467063129,24.740524,661,0,105,27021597920011982,43.66176081083393,2015.5,-51.836,-27.3391,126.00120619932957,12.9807,12.300461,12.648243,9.174,11.121874,nan,11.5654955,10.780996,11.5745,9.77,9.036,sdss_psfmag_gaia,497069781.0,APOGEE,mwm_snc_100pc_apogee


In [11]:
np.unique(best_detail_tab['design_id'])

12780
12781
12787
12788
12798
12853
12875
13135
13467


In [12]:
best_detail_tab.write('best_field_detailed_targets_cool_star.fits')